## Install and import packages

In [1]:
# !pip install soundfile
# !pip install pydub
# !pip install SpeechRecognition
# !pip install openai --upgrade

In [2]:
import os
import csv
import wave
import soundfile
import requests
import pandas as pd
from datetime import timedelta
from pydub import AudioSegment
import speech_recognition as sr
from transformers import pipeline
from openai import AzureOpenAI
from dotenv import load_dotenv
from tqdm import tqdm


# Load environment variables from .env file
load_dotenv()

# Get the Hugging Face API key from the environment variables
HF_API_KEY = os.environ["HF_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

/Users/kw/Documents/py_projects/canto-text/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 
#### Setup Device (CUDA, MPS, CPU)

In [3]:
import torch

# Determine the best available device (CUDA if available, otherwise MPS, then CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [4]:
input_m4a_audio = "input_audio/sample_20mins.m4a"

---
## Convert audio to wav format

In [5]:
from pydub import AudioSegment
import os

def convert_audio_to_wav(input_file):
    """Convert .m4a file to .wav format."""
    # Load the .m4a file
    audio = AudioSegment.from_file(input_file, format="m4a")
    output_file = os.path.join("converted_audio", os.path.basename(input_file).replace(".m4a", ".wav"))
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    audio.export(output_file, format="wav")
    
    return output_file

In [6]:
convert_audio_to_wav(input_m4a_audio)

'converted_audio/sample_20mins.wav'

---
## Audio Chunking
### using pyannote

In [7]:
import os
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import medfilt

def split_audio_by_silence(audio_path, output_dir, chunk_duration=300, min_silence_duration=0.5, 
                           silence_threshold=-40, padding=0.2):
    """
    Split an audio file into chunks based on silence detection to avoid splitting in the middle of sentences.
    
    Parameters:
    - audio_path: Path to the input audio file
    - output_dir: Directory to save the output chunks
    - chunk_duration: Target duration for each chunk in seconds (default: 300s = 5 minutes)
    - min_silence_duration: Minimum duration of silence to consider as a potential split point (seconds)
    - silence_threshold: Threshold in dB for determining silence
    - padding: Padding time in seconds to add to each chunk to ensure overlap
    
    Returns:
    - List of paths to the generated audio chunks
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the audio file with pydub
    audio = AudioSegment.from_file(audio_path)
    
    # Convert to mono if stereo for easier processing
    if audio.channels > 1:
        audio = audio.set_channels(1)
    
    # Get the duration in milliseconds
    total_duration_ms = len(audio)
    
    # Convert chunk duration to milliseconds
    chunk_duration_ms = chunk_duration * 1000
    
    # Load the audio file with librosa for silence detection
    y, sr = librosa.load(audio_path, sr=None)
    
    # Compute the RMS energy
    rms = librosa.feature.rms(y=y, frame_length=int(sr * 0.025), hop_length=int(sr * 0.01))[0]
    
    # Convert to dB
    rms_db = 20 * np.log10(rms + 1e-10)
    
    # Apply median filtering to smooth the energy curve
    rms_db_smoothed = medfilt(rms_db, kernel_size=11)
    
    # Find silence regions (below threshold)
    silence_mask = rms_db_smoothed < silence_threshold
    
    # Convert to time (seconds)
    times = librosa.times_like(rms_db_smoothed, sr=sr, hop_length=int(sr * 0.01))
    
    # Find continuous silence regions
    silence_regions = []
    in_silence = False
    start_time = 0
    
    for i, is_silent in enumerate(silence_mask):
        if is_silent and not in_silence:
            # Start of a silence region
            start_time = times[i]
            in_silence = True
        elif not is_silent and in_silence:
            # End of a silence region
            end_time = times[i]
            silence_duration = end_time - start_time
            if silence_duration >= min_silence_duration:
                silence_regions.append((start_time, end_time))
            in_silence = False
    
    # Add the last silence region if we're still in silence at the end
    if in_silence:
        end_time = times[-1]
        silence_duration = end_time - start_time
        if silence_duration >= min_silence_duration:
            silence_regions.append((start_time, end_time))
    
    # Sort silence regions by their start time
    silence_regions.sort(key=lambda x: x[0])
    
    # Determine the split points
    split_points = []
    current_chunk_end_ms = chunk_duration_ms
    
    # Find the closest silence region to each target chunk end
    for i in range(len(silence_regions)):
        silence_start_ms = silence_regions[i][0] * 1000
        silence_end_ms = silence_regions[i][1] * 1000
        
        # If this silence region is close to our target chunk end
        if abs(silence_start_ms - current_chunk_end_ms) < chunk_duration_ms * 0.2:
            split_points.append(silence_start_ms)
            current_chunk_end_ms += chunk_duration_ms
        
        # If we've already passed our target chunk end, use this silence region
        elif silence_start_ms > current_chunk_end_ms:
            split_points.append(silence_start_ms)
            current_chunk_end_ms = silence_start_ms + chunk_duration_ms
    
    # Add the start of the audio as the first split point
    split_points = [0] + split_points
    
    # Add the end of the audio as the last split point
    if split_points[-1] < total_duration_ms:
        split_points.append(total_duration_ms)
    
    # Generate chunks
    chunk_paths = []
    
    for i in range(len(split_points) - 1):
        # Calculate start and end times with padding
        start_ms = max(0, split_points[i] - padding * 1000)
        end_ms = min(total_duration_ms, split_points[i + 1] + padding * 1000)
        
        # Extract the chunk
        chunk = audio[start_ms:end_ms]
        
        # Generate the output filename
        output_filename = f"chunk_{i+1:03d}.wav"
        output_path = os.path.join(output_dir, output_filename)
        
        # Export the chunk
        chunk.export(output_path, format="wav")
        chunk_paths.append(output_path)
        
        # Print progress
        print(f"Chunk {i+1}/{len(split_points)-1} - Duration: {(end_ms-start_ms)/1000:.2f}s")
    
    return chunk_paths


In [8]:
raw_wav_file = input_m4a_audio.replace('input_audio', 'converted_audio').replace('.m4a', '.wav')
file_name = input_m4a_audio.split('/')[-1].split('.')[0]

In [9]:
split_audio_by_silence(raw_wav_file, f"chunked_audio/{file_name}", chunk_duration=80)

Chunk 1/18 - Duration: 78.62s
Chunk 2/18 - Duration: 68.57s
Chunk 3/18 - Duration: 96.01s
Chunk 4/18 - Duration: 86.13s
Chunk 5/18 - Duration: 57.94s
Chunk 6/18 - Duration: 79.54s
Chunk 7/18 - Duration: 81.10s
Chunk 8/18 - Duration: 98.52s
Chunk 9/18 - Duration: 69.71s
Chunk 10/18 - Duration: 142.17s
Chunk 11/18 - Duration: 111.12s
Chunk 12/18 - Duration: 118.00s
Chunk 13/18 - Duration: 73.12s
Chunk 14/18 - Duration: 93.91s
Chunk 15/18 - Duration: 99.56s
Chunk 16/18 - Duration: 84.22s
Chunk 17/18 - Duration: 61.05s
Chunk 18/18 - Duration: 63.22s


['chunked_audio/sample_20mins/chunk_001.wav',
 'chunked_audio/sample_20mins/chunk_002.wav',
 'chunked_audio/sample_20mins/chunk_003.wav',
 'chunked_audio/sample_20mins/chunk_004.wav',
 'chunked_audio/sample_20mins/chunk_005.wav',
 'chunked_audio/sample_20mins/chunk_006.wav',
 'chunked_audio/sample_20mins/chunk_007.wav',
 'chunked_audio/sample_20mins/chunk_008.wav',
 'chunked_audio/sample_20mins/chunk_009.wav',
 'chunked_audio/sample_20mins/chunk_010.wav',
 'chunked_audio/sample_20mins/chunk_011.wav',
 'chunked_audio/sample_20mins/chunk_012.wav',
 'chunked_audio/sample_20mins/chunk_013.wav',
 'chunked_audio/sample_20mins/chunk_014.wav',
 'chunked_audio/sample_20mins/chunk_015.wav',
 'chunked_audio/sample_20mins/chunk_016.wav',
 'chunked_audio/sample_20mins/chunk_017.wav',
 'chunked_audio/sample_20mins/chunk_018.wav']

---

## Speech (Cantonese) to Text (Spoken Cantonese)
### Method 1: using Google Gemini 1.5 pro

In [10]:
def get_all_file_paths(directory):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):  # Make sure to only include .wav files
                file_paths.append(os.path.join(root, file))
    
    # Sort the file paths based on the numeric part of the filename
    file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1]))
    
    return file_paths

In [11]:
file_name

'sample_20mins'

In [12]:
audio_chunk_paths = get_all_file_paths(f'chunked_audio/{file_name}')
audio_chunk_paths

['chunked_audio/sample_20mins/chunk_001.wav',
 'chunked_audio/sample_20mins/chunk_002.wav',
 'chunked_audio/sample_20mins/chunk_003.wav',
 'chunked_audio/sample_20mins/chunk_004.wav',
 'chunked_audio/sample_20mins/chunk_005.wav',
 'chunked_audio/sample_20mins/chunk_006.wav',
 'chunked_audio/sample_20mins/chunk_007.wav',
 'chunked_audio/sample_20mins/chunk_008.wav',
 'chunked_audio/sample_20mins/chunk_009.wav',
 'chunked_audio/sample_20mins/chunk_010.wav',
 'chunked_audio/sample_20mins/chunk_011.wav',
 'chunked_audio/sample_20mins/chunk_012.wav',
 'chunked_audio/sample_20mins/chunk_013.wav',
 'chunked_audio/sample_20mins/chunk_014.wav',
 'chunked_audio/sample_20mins/chunk_015.wav',
 'chunked_audio/sample_20mins/chunk_016.wav',
 'chunked_audio/sample_20mins/chunk_017.wav',
 'chunked_audio/sample_20mins/chunk_018.wav']

In [13]:
import google.generativeai as genai

# Gemini transcription and diarization function
def transcribe_with_gemini(audio_chunk_paths, api_key):
    try:
        print("Transcribing with Gemini Pro 1.5...")
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')
        
        # Create prompt for diarization and Cantonese transcription
        prompt = """
        Transcribe this Cantonese audio while eliminating filler words and repeating words.
        - The audio contains a conversation between multiple speakers but 
        """
        
        transcriptions = []
        for i, chunk_path in enumerate(tqdm(audio_chunk_paths, desc="Transcribing chunks")):
            # Upload audio file with specified MIME type
            f = genai.upload_file(chunk_path, mime_type='audio/wav')
            
            # Generate using multimodal capabilities
            response = model.generate_content([prompt, f])
            transcriptions.append(response.text)
        
        return "".join(transcriptions)
    except Exception as e:
        print(f"Error with Gemini transcription: {str(e)}")
        return None

In [14]:
# Test with a single file
# audio_chunk_paths = ["converted_audio/sample_audio_1min.wav"]
gemini_output = transcribe_with_gemini(audio_chunk_paths, GEMINI_API_KEY)
print(gemini_output)

Transcribing with Gemini Pro 1.5...


Transcribing chunks: 100%|██████████| 18/18 [05:34<00:00, 18.61s/it]

關於教師發展課程，好開心，一點啦啦申請，只有你，因為截止啦，其實截止啦，因為1月31號嘛，我都唔記得咗，今日今日27號，發現邊問題喎，只有你，嗰年1月1號，去到，其實年年都有嘅，不過我唔記得咗多次，年年都有呢個申請，係嗰年1月1號，去到今年的1月31號喎，之前你已經完成晒成個Course，然後有單有證書，可以申請，例如，Course Cert，你未讀㗎？讀咗，讀咗攞咗證書，攞咗。有收據？有收據，正本證書？係，辦妥先，好，申請咗先，好，申請我幫你，辦妥，即時幫你。因為申請咗嘛，因為過咗如果唔再申請呢，就點呀？就過咗囉。因為佢下個年度都會再繼續，就係下年嘅2025年1月1號囉，去到，因為你收據之前，係，咁就趁聽日囉，係，你就即時可以追呀嘛。應該仲有成個月喎，哦，係啦係啦，收到已經讀晒啦，係呀係呀，所以就申請啦，例如例如Cert，繼續申請下呢個， 삐唔便宜咩？初初初初…唔係啫… 講嘅事實呀，係唔係先？大家都係個好嘅Idea，因為同我哋實際有關嘅，等我講錯，其實我哋做嘢出發點係嗰啲牌照嘅嘢。 計劃好啲。 你哋好啦。 好呀。 除非做好長，就可以圍返啲。 你咪… 你話… 其他老師出發點即知道嘅。 係啦，睇你自己小朋友，我哋老師，我連做其他… 佢咪做咗，其他人做其他。 不過後來呢就開始… 之嘛。 因為佢哋話嗰會，佢哋話… 都講… 其實… 時數時數… 但其他老師，佢哋之前嗰啲老師嘅，真實情況係咁㗎嘛。但有咩情況之下呢，其實唔到時，其實長… 真係好唔開心㗎喎… 係啦，一千零一夜、二千零一夜、三千零一夜咩？係一千零一個啫，但所以呢，我哋其他小朋友，咁你真嘅，唔係講笑，係要呢… 其實唔係嗰陣時，你哋又…買…唔係，我今日唔講，大家個喎。 係喇，咁如果唔係呢，老師你覺得呢？  我哋呢就，我哋夾，其實呢，夾咗幾次呢，目前讀數寫呢，都唔係幾好咁解。 唔得㗎，我哋唔，我哋就因為嗰陣時... 你哋裡面應該其他全部都有㗎應該。 咩，呢嗰... 大幾個嘅，佢哋可能根本... 份數份數呢... 所以... 唔得唔得，我哋嗰啲小朋友嚟㗎。 小朋友。 係，咁我自己... 咁我... 快啲令到自己升價好啲，咁個個都希望能夠可以幫到小朋友嘛，真係好難㗎喎呢啲。 係呀，咁所以大家畀啲... 嗰啲幾千呀？ 幾千蚊嚟嘅。 嗯。 係，幾千蚊嚟嘅。 嗯。 吓，就咪請人過嚟到校教喇，人數唔夠。 係唔夠，你哋得嗰一二三四五六七

### Define Generic Gemini prompting function

In [15]:
import google.generativeai as genai

def gemini_prompt_call(message, api_key, prompt, temperature=0.3, top_p=0.8):
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')

        prompt_parts = [prompt+'\n', message]

        generation_config = genai.GenerationConfig(
                            temperature=temperature,
                            top_p=top_p,
                            )  

        response = model.generate_content(prompt_parts, generation_config=generation_config)
        return response.text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Add punctuation and refine the Spoken Cantonese Text
Using Gemini 1.5 pro

In [16]:
def refine_cantonese(message, api_key):
    prompt = """
            You are an expert in Cantonese. Please perform the following two steps and return the revised text.
            1. Correct potential inaccuracies in the Cantonese text in Hong Kong style. For example, in Hong Kong, text such as '噉', should be written as '咁'.
            2. Refine punctuation to the text.
            3. Remove repeated and unnecessary words.  For example, '係嘅係嘅係嘅' or '啱嘅啱嘅啱嘅' can be replaced with '係'.
            """
    return gemini_prompt_call(message, api_key, prompt)

In [17]:
gemini_output

'關於教師發展課程，好開心，一點啦啦申請，只有你，因為截止啦，其實截止啦，因為1月31號嘛，我都唔記得咗，今日今日27號，發現邊問題喎，只有你，嗰年1月1號，去到，其實年年都有嘅，不過我唔記得咗多次，年年都有呢個申請，係嗰年1月1號，去到今年的1月31號喎，之前你已經完成晒成個Course，然後有單有證書，可以申請，例如，Course Cert，你未讀㗎？讀咗，讀咗攞咗證書，攞咗。有收據？有收據，正本證書？係，辦妥先，好，申請咗先，好，申請我幫你，辦妥，即時幫你。因為申請咗嘛，因為過咗如果唔再申請呢，就點呀？就過咗囉。因為佢下個年度都會再繼續，就係下年嘅2025年1月1號囉，去到，因為你收據之前，係，咁就趁聽日囉，係，你就即時可以追呀嘛。應該仲有成個月喎，哦，係啦係啦，收到已經讀晒啦，係呀係呀，所以就申請啦，例如例如Cert，繼續申請下呢個， 삐唔便宜咩？初初初初…唔係啫… 講嘅事實呀，係唔係先？大家都係個好嘅Idea，因為同我哋實際有關嘅，等我講錯，其實我哋做嘢出發點係嗰啲牌照嘅嘢。 計劃好啲。 你哋好啦。 好呀。 除非做好長，就可以圍返啲。 你咪… 你話… 其他老師出發點即知道嘅。 係啦，睇你自己小朋友，我哋老師，我連做其他… 佢咪做咗，其他人做其他。 不過後來呢就開始… 之嘛。 因為佢哋話嗰會，佢哋話… 都講… 其實… 時數時數… 但其他老師，佢哋之前嗰啲老師嘅，真實情況係咁㗎嘛。但有咩情況之下呢，其實唔到時，其實長… 真係好唔開心㗎喎… 係啦，一千零一夜、二千零一夜、三千零一夜咩？係一千零一個啫，但所以呢，我哋其他小朋友，咁你真嘅，唔係講笑，係要呢… 其實唔係嗰陣時，你哋又…買…唔係，我今日唔講，大家個喎。 係喇，咁如果唔係呢，老師你覺得呢？  我哋呢就，我哋夾，其實呢，夾咗幾次呢，目前讀數寫呢，都唔係幾好咁解。 唔得㗎，我哋唔，我哋就因為嗰陣時... 你哋裡面應該其他全部都有㗎應該。 咩，呢嗰... 大幾個嘅，佢哋可能根本... 份數份數呢... 所以... 唔得唔得，我哋嗰啲小朋友嚟㗎。 小朋友。 係，咁我自己... 咁我... 快啲令到自己升價好啲，咁個個都希望能夠可以幫到小朋友嘛，真係好難㗎喎呢啲。 係呀，咁所以大家畀啲... 嗰啲幾千呀？ 幾千蚊嚟嘅。 嗯。 係，幾千蚊嚟嘅。 嗯。 吓，就咪請人過嚟到校教喇，人數唔夠。 係唔夠，你哋得嗰一二三四五六

In [18]:
refined_spoken_text = refine_cantonese(gemini_output, GEMINI_API_KEY)

In [19]:
refined_spoken_text

"關於教師發展課程，好開心，只需申請一次，因為截止日期為1月31號。今日27號，發現有個問題，就是你1月1號至31號期間，可以申請。其實年年都有，不過我唔記得咗幾次。之前你已完成整個課程，並有證書和收據，可以申請，例如Course Cert。正本證書？係，辦妥先。好，申請咗先。好，我幫你申請，辦妥，即時幫你。因為如果過咗期限唔再申請，就過咗期。下年度，即2025年1月1號，都會再繼續。咁就趁聽日，即時追。應該仲有成個月。收到，已經讀晒，所以就申請，例如Cert，繼續申請。삐唔便宜咩？唔係，講嘅係事實。係唔係先？大家都係個好idea，因為同我哋實際有關，等我講錯，其實我哋做嘢出發點係嗰啲牌照嘅嘢。計劃好啲。你哋好啦。好呀。除非做好長，就可以圍返啲。你話其他老師出發點即知道。係啦，睇你自己小朋友，我哋老師，我連做其他…佢咪做咗，其他人做其他。不過後來就開始…之嘛。因為佢哋話…都講…其實…時數時數…但其他老師，佢哋之前嗰啲老師，真實情況係咁㗎嘛。但有咩情況之下呢，其實唔到時，其實長…真係好唔開心㗎喎…係啦，一千零一夜，唔係一千零一個啫，但所以呢，我哋其他小朋友，咁你真嘅，唔係講笑，係要…其實唔係嗰陣時，你哋又…買…唔係，我今日唔講，大家個喎。係喇，咁如果唔係呢，老師你覺得呢？我哋呢就，我哋夾，其實呢，夾咗幾次呢，目前讀數寫呢，都唔係幾好。唔得㗎，我哋就因為嗰陣時…你哋裡面應該其他全部都有㗎應該。咩，呢個…大幾個嘅，佢哋可能根本…份數份數呢…所以…唔得唔得，我哋嗰啲小朋友嚟㗎。小朋友。係，咁我自己…咁我…快啲令到自己升價好啲，咁個個都希望能夠幫到小朋友嘛，真係好難㗎喎呢啲。係呀，咁所以大家畀啲…嗰啲幾千呀？幾千蚊嚟嘅。嗯。係，幾千蚊嚟嘅。嗯。吓，就咪請人過嚟到校教喇，人數唔夠。係唔夠，你哋得八個人啫。係呀。唔夠喎。係呀。真係自己夾，夾咗幾次都唔…出咗幾次，係咪好快㗎嘛，係。唔係我話，即係今日咁啱有，兩年時間。兩年時間。係，呢啲嘢好，教我嗰…係呀如果唔係好，咁如果唔係咁安排用兩年時間慢慢處理咗佢，呢個係需要處理㗎嘛。咁呢就慳返錢，咁你哋應該，其實就嗰音樂創嚟㗎，就係，啊，音樂教育法咁樣。都成幾千蚊㗎咁樣，都好囉幾個。\n\n你話畀佢，我哋幫你send到，你自己打啦，係咪呀？百幾頁書。如果真係有呢啲嘢，你哋個別學校嘅教育有關，呢個教師法應該用得着㗎。好唔好啊？唔好嘥咗佢，因為呢

## Speech (Cantonese) to Text (Written Chinese)
Using Gemini 1.5 pro

In [20]:
def speech_to_written(message, api_key):
    prompt = """
            Convert the following spoken Cantonese text into standard written Chinese. Maintain grammatical correctness and coherence while ensuring the meaning is preserved. Use formal written Chinese (書面語) rather than colloquial Cantonese (口語).
            """
    return gemini_prompt_call(message, api_key, prompt)

In [21]:
raw_written = speech_to_written(refined_spoken_text, GEMINI_API_KEY)

In [22]:
raw_written

"關於教師發展課程，截止日期為1月31日，只需申請一次即可。現已1月27日，課程申請時間為1月1日至31日。該課程每年均有提供，之前若已完成課程並持有證書及收據（例如Course Cert正本），即可申請。請盡快申請，以免逾期。下年度（2025年1月1日）課程將繼續開放。已完成課程並持有證書（例如Cert）者，請立即申請。此課程與我們的牌照事務息息相關，是一個好主意，值得好好規劃。關於其他老師的想法，則需視乎個人情況。關於時數問題，真實情況與之前的老師們所經歷的相似。但具體情況需視乎實際情況而定，若課程時間過長，則令人不悅。關於小朋友的學習，例如顏色辨識，需根據個別情況提供協助，切勿妄下判斷。\n\n關於課程費用（幾千元），可由學校統一申請，然後再發放給各位老師。申請時需提供申請書、圖原、單據及正本證書。關於網絡安全及網上發情問題，總部已有要求，但目前尚未收到正式文件。相關軟體已置於教師電腦中，請勿用於讀寫，以免損壞。如有問題，可參考相關書籍及Q&A資料。關於聯校及香港世界活動，比賽將於4月舉行，具體日期暫定為4月12日，地點為油麻地城。比賽將分初賽及決賽，K1、K2、K3均可參加。比賽旨在提升學生的能力，而非增加壓力，請各位老師多向學生講解常識。美術比賽將於6月7日舉行，地點為某中學。\n\n關於層次問題，請參考之前所述。關於升水及差別，請根據個別情況處理。請為六月份的活動做好準備。關於文章及證書問題，請參考已發放的資料。關於機器問題，請多加處理。關於字典問題，請參考相關資料。關於溝通問題，已更換設備，並提供卡片供學生使用，以提升技巧。關於書包及卡片問題，請各位老師多加留意，避免學生遺失。關於餐具問題，請妥善保管，避免遺失。關於電子產品及餐具共用問題，請各位老師提醒學生保持自律。關於禮物問題，請參考標準禮物清單，並填寫相關資料。關於評分問題，請參考相關規定。關於輔導點問題，將另行通知。關於會議及功課安排，將於下個月開始。關於教材教具，將於9月提供具體日期。關於K1、K2、K3的主題安排，下年度將統一開始及結束，以便處理。關於家長活動，請參考已發放的資料。關於親子活動，請參考相關規定，並將特別情況加入其中。關於假期安排，請自行參考。關於K1、K2、K3的活動安排，請參考相關表格。關於三罷問題，將保持彈性安排。關於手寫及三個字問題，請盡力完成。關於活動安排，請參考相關

## Refine Written Chinese (smoother sentence)
Using Gemini 1.5 pro

In [23]:
def refine_writte_text(message, api_key):
    prompt = """
            Please refine the following Chinese text by making the sentence structure smoother and more coherent. 
            Reduce excessive punctuation, clarify ambiguous parts, and ensure natural readability while preserving the original meaning.
            """
    return gemini_prompt_call(message, api_key, prompt, temperature=1)

In [24]:
refined_written = refine_writte_text(raw_written, GEMINI_API_KEY)

In [25]:
refined_written

'教師發展課程申請截止日期為1月31日，只需申請一次。目前已1月27日，申請時間為1月1日至31日。此課程每年均有提供，若曾完成課程並持有證書及收據（例如Course Cert正本），即可申請。請盡快申請，以免逾期。下年度（2025年1月1日）課程亦將開放申請。已完成課程並持有證書（例如Cert）者，請立即申請。此課程與本校牌照事務息息相關，值得各位老師妥善規劃及參與。其他老師的想法則視乎個人情況。\n\n關於課程時數，與往年老師的經驗相似，但具體時數仍需視實際情況而定。若課程時間過長，可能會造成不便。\n\n關於小朋友的學習，例如顏色辨識，請根據個別情況提供協助，切勿妄下判斷。\n\n課程費用（約數千元）可由學校統一申請後發放給各位老師。申請時需提交申請書、圖像原稿、單據及證書正本。\n\n關於網絡安全及網上不當言論問題，總部已有相關要求，但正式文件尚未下達。教師電腦中已安裝相關軟體，請勿用於讀寫操作，以免損壞。如有疑問，請參考相關書籍及Q&A資料。\n\n聯校及香港世界活動比賽將於4月12日（暫定）在油麻地城舉行，分初賽及決賽，K1、K2、K3學生均可參加。比賽旨在提升學生能力，而非增加壓力，請各位老師多向學生講解相關常識。美術比賽將於6月7日在某中學舉行。\n\n關於層次問題，請參考先前說明。升水及差別問題請根據個別情況處理。請各位老師為六月份的活動做好準備。文章及證書問題請參考已發放的資料。機器問題請多加處理。字典問題請參考相關資料。\n\n為提升學生溝通技巧，已更換通訊設備，並提供卡片供學生使用。請各位老師多加留意學生的書包及卡片，避免遺失。餐具亦請妥善保管，避免遺失。請提醒學生妥善使用電子產品及餐具，避免共用。\n\n禮物發放請參考標準禮物清單，並填寫相關資料。評分請參考相關規定。輔導點事宜將另行通知。會議及功課安排將於下個月開始。教材教具的具體提供日期將於9月公布。\n\n下年度K1、K2、K3的主題安排將統一開始及結束，以便管理。家長活動請參考已發放的資料。親子活動請參考相關規定，並將特殊情況納入考量。假期安排請自行參考。K1、K2、K3的活動安排請參考相關表格。關於「三罷」問題，將保持彈性安排。手寫及三個字的要求請盡力完成。活動安排請參考相關資訊，並可使用手機發佈至Facebook。\n\n\nKey changes:\n\n* **Reduced

## Summarize using Gemini 1.5 Pro

In [26]:
def summarize_refined_written(message, api_key, sum_lvl):
    prompt = f"""Summarize the given Chinese text in Traditional Chinese (zh) at summarization level {sum_lvl}, where 1 is the most concise and 5 is the most detailed, and 0 is in bullet points with sectioned format. Retain key information and context while ensuring clarity, coherence, and fluency."""
    return gemini_prompt_call(message, api_key, prompt)

In [27]:
def summarize_refined_written(message, api_key, sum_lvl, bullet_points=False):
    if bullet_points:
        prompt = f"""Summarize the given Chinese text and output in Traditional Chinese (zh) markdown format with bullet points and sectioned format. Use detail level {sum_lvl}, where 1 is the most concise and 5 is the most detailed. Retain key information and context while ensuring clarity, coherence, and fluency."""
    else:
        prompt = f"""Summarize the given Chinese text and output in Traditional Chinese (zh) at summarization level {sum_lvl}, where 1 is the most concise and 5 is the most detailed. Retain key information and context while ensuring clarity, coherence, and fluency."""
    
    return gemini_prompt_call(message, api_key, prompt)

In [28]:
# Test
summarized_text = summarize_refined_written(refined_written, GEMINI_API_KEY, sum_lvl=2, bullet_points=True)

In [29]:
print(summarized_text)

## 教師發展及學校事務概要

### 教師發展課程

* 申請截止日期：1月31日 (只需申請一次，目前已1月27日，申請時間為1月1日至31日)
* 資格：曾完成課程並持有證書及收據 (例如 Course Cert 正本)
* 2025年度課程將於2025年1月1日開放申請
* 課程與學校牌照事務相關，建議參與
* 課程時數：與往年相若，視實際情況而定
* 課程費用：約數千元，由學校統一申請後發放，需提交申請書、圖像原稿、單據及證書正本

### 學生學習及活動

* 顏色辨識等學習：根據個別學生情況提供協助
* 聯校及香港世界活動比賽：
    * 日期：4月12日 (暫定)
    * 地點：油麻地城
    * 賽制：分初賽及決賽
    * 參加對象：K1、K2、K3 學生
    * 目標：提升學生能力
* 美術比賽：
    * 日期：6月7日
    * 地點：某中學
* 六月份活動：請老師做好準備
* 通訊設備更新：已更換通訊設備並提供卡片，請留意學生書包及卡片避免遺失
* 餐具：請妥善保管，避免遺失，並提醒學生避免共用
* 電子產品使用：提醒學生妥善使用，避免共用

### 行政及其他事項

* 網絡安全及網上不當言論：總部已有相關要求，教師電腦已安裝相關軟體，請勿用於讀寫操作
* 層次、升水及差別問題：請參考先前說明及根據個別情況處理
* 文章及證書問題：請參考已發放資料
* 機器問題：請多加處理
* 字典問題：請參考相關資料
* 禮物發放：參考標準禮物清單並填寫相關資料
* 評分：參考相關規定
* 輔導點事宜：另行通知
* 會議及功課安排：下個月開始
* 教材教具提供日期：9月公布
* 下年度 K1、K2、K3 主題：統一開始及結束
* 家長活動：參考已發放資料
* 親子活動：參考相關規定，並考慮特殊情況
* 假期安排：自行參考
* K1、K2、K3 活動安排：參考相關表格
* 「三罷」問題：保持彈性安排
* 手寫及三個字要求：盡力完成
* 活動安排資訊發佈：可使用手機發佈至 Facebook

